In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import os
books = pd.read_csv(os.path.join(path, "books.csv"))
display(books)

# Analisando os dados faltantes

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = plt.axes()
sns.heatmap(books.isna().transpose(), cbar=False, ax=ax)
ax.set_title("Missing values in dataset")
plt.ylabel("Missing values")
plt.show()

In [ ]:
# Criando novas variáveis
import numpy as np

books["missing_description"] = np.where(books["description"].isna(), 1, 0)
books["age_of_book"] = 2025 - books["published_year"]

In [ ]:
#selecionamos nossas colunas de interesse

columns_of_interest = ["num_pages", "age_of_book", "missing_description", "average_rating"]
correlation_matrix = books[columns_of_interest].corr(method="spearman") #bom quanto vc está lidando com dados discretos
sns.set_theme(style="white")
plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", cbar_kws={"label": "Correlation coefficient"})
heatmap.set_title("Correlation matrix")

In [ ]:
#Como a perda de descrição não tem correlação com nenhum outro atributo, podemos remover os livros sem descrição, assim como remover dados faltantes das outras variáveis

#Primeiro checamos quanto perderíamos de dados se removessemos os livros sem description, num_pages, average_rating e published_year

book_missing = books[(books["description"].isna()) | (books["num_pages"].isna()) | (books["average_rating"].isna()) | (books["published_year"].isna())]

print("Percentage of data lost if we remove rows with missing values:", (len(book_missing) / len(books)) * 100)
#como o valor é baixo, podemos remover os dados faltantes

books_filtered = books.dropna(subset=["description", "num_pages", "average_rating", "published_year"])
display(books_filtered)


In [ ]:
# Como vamos recomendar os livros pela classificação, precisamos checar como esta a distribuição de  categories	

books_filtered["categories"].value_counts().reset_index().sort_values("count", ascending=False).head(40).set_index("categories").plot(kind="bar", figsize=(10, 6))

# Percebemos que a distribuição não é normal e precisamos ver como vamos normalizar esses dados

In [ ]:
#No dataframe, também percebe-se que na descrição, que também será utilizada pra treinar o modelo de LLM, temos muitos valores não informativos, com informações curtas ou sem utilidade

#Vamos então filtrar pelo número de palavras:

books_filtered["description_word_count"] = books_filtered["description"].str.split().str.len()

#vamos ver num histograma com intervalos de 10 palavras

books_filtered["description_word_count"].hist(bins=range(0, 1000, 10), figsize=(10, 6))


In [ ]:
# Agora vamos checar o que são essas descrições de até 10 palavras

books_filtered[books_filtered["description_word_count"] < 10]["description"].head(10)

#não são descritivas

In [ ]:
# Agora até 20 palavras

display(books_filtered[(books_filtered["description_word_count"] <= 20) & (books_filtered["description_word_count"] >= 10)]["description"].head(10))

#Talvez não tenha como ver bem aqui, mas ainda não está muito descritivo, mas se baixa-se num csv daria para verificar

In [ ]:
# Agora até 30 palavras

display(books_filtered[(books_filtered["description_word_count"] <= 30) & (books_filtered["description_word_count"] >= 20)]["description"].head(10))

#Agora sim, temos descrições mais descritivas

In [ ]:
#Aumentamos um pouco por garantia

book_no_missing_words = books_filtered[books_filtered["description_word_count"] >= 25]
display(book_no_missing_words)

In [ ]:
# Agora vamos criar uma variável que junta o título com o subtítulo

import numpy as np

book_no_missing_words["title_subtitle"] = np.where(
    book_no_missing_words["subtitle"].isna(), 
    book_no_missing_words["title"],
    book_no_missing_words[["title", "subtitle"]].astype(str).agg(": ".join, axis=1)
    )

display(book_no_missing_words[["title_subtitle"]].head(10))



In [ ]:
# Vamos também conectar os tags com as descrições

book_no_missing_words["tag_description"] = book_no_missing_words[["isbn13", "description"]].astype(str).agg(" ".join, axis=1)
display(book_no_missing_words["tag_description"].head(10))

In [ ]:
#Vamos descartar as colunas que não vamos usar

display(book_no_missing_words.columns)

(
    book_no_missing_words
        .drop(columns=["subtitle", "missing_description", "age_of_book"])
        .to_csv("books_cleaned.csv", index=False)
)
